**This is the most updated LDA Notebook**
This notebook does the following:
*   Preprocess the corpus. --> after adding the papers with no XMLs and removing the essays
*   Perform LDA on the corpus with 10 and 12 topics
*   Visualize the topics, calculate perplexities and coherence scores
*   Make spreadsheets of the results:  these spreadsheets are in the folder "Journal_Analysis/best_model/No_Essays"

In [ ]:
pip install pyLDAvis

     |████████████████████████████████| 1.7MB 5.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.3MB 424kB/s 
     |████████████████████████████████| 9.9MB 23.7MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136870 sha256=512bca54adeea9b350fb0a1ecbebb44c859fe212f1e76d148ef28b64c0c71e2e
  Stored in directory: /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2
Successfully built pyLDAvis
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0

In [ ]:
pip install --upgrade gensim

     |████████████████████████████████| 23.9MB 1.3MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


Restart Kernel

In [ ]:
import nltk
import numpy as np
import pandas as pd
from nltk import word_tokenize
import re
import seaborn as sns
import matplotlib.pyplot as plt
import smart_open
from gensim.models import LdaModel
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import spacy


/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecate

In [ ]:
import gensim
gensim.__version__

'4.0.1'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Journal_Analysis/"

In [ ]:
def extract_documents(folder_name):
  import os
  docs = []
  allfiles = []
  for entry in os.scandir(folder_name):
      if entry.path.endswith(".txt") and entry.is_file():
          path_paper = entry.path
          allfiles.append(path_paper)
          if os.stat(path_paper).st_size == 0:
              continue
          with open(path_paper) as f:
            text = f.read()
            docs.append(text)
  return docs, allfiles

In [ ]:
docs, allfiles = extract_documents(path + 'allpapers_NoEssays')

In [ ]:
print(len(docs))
print(docs[0][:500])

220
Specialization and Career Dynamics
Evidence from the Indian Administrative Service
In this article, we attempt to resolve the tension between two conflicting views on the role of specialization in workers’ careers. Some scholars argue that specialization is a net benefit that allows workers to get ahead, while others argue that broad experience across several domains is the only way to be truly exceptional. We use rich longitudinal data from 1974 to 2008 on the careers of Indian Administrative S


In [ ]:
allfiles[0]

'/content/drive/MyDrive/Journal_Analysis/allpapers_NoEssays/10.1177_0001839213486759.txt'

In [ ]:
print(docs[0][:500])

Specialization and Career Dynamics
Evidence from the Indian Administrative Service
In this article, we attempt to resolve the tension between two conflicting views on the role of specialization in workers’ careers. Some scholars argue that specialization is a net benefit that allows workers to get ahead, while others argue that broad experience across several domains is the only way to be truly exceptional. We use rich longitudinal data from 1974 to 2008 on the careers of Indian Administrative S


In [ ]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.
    #docs[idx] = docs[idx].lower()  # Convert to lowercase.

# Remove numbers, but not words that contain numbers.
docs = [[token.lower() for token in doc] for doc in docs]
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are less than 3 characters.
docs = [[token for token in doc if len(token) > 2] for doc in docs]



In [ ]:
# Lemmatize the documents.
import nltk
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#Filter out stop words
nltk.download('stopwords')
en_stop = nltk.corpus.stopwords.words('english')
print(len(en_stop))
en_stop.extend(["organization", "organizational", "organizing", "would", "could", "result", "study", "model", "likely", "effect", "effected", "variable", "variables", "measure", "include", "suggest", "first", "level", "research"])
print(len(en_stop))
"organization" in en_stop

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
179
198


True

In [ ]:
en_stop

In [ ]:
docs = [[token for token in doc if token not in en_stop] for doc in docs]   #remove stopwords

In [ ]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
            #print("yes")

In [ ]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print(len(dictionary))

# Filter out words that occur less than 3 documents, or more than 70% of the documents.
dictionary.filter_extremes(no_below=3, no_above=0.7)
print(len(dictionary))


27852
12912


In [ ]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [ ]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 12912
Number of documents: 220


In [ ]:
def modify_string(probability_string):
  prob = [float(x.split("*")[0]) for x in probability_string.split("+")]
  words = [x.split("*")[1] for x in probability_string.split("+")]
  return prob,words

In [ ]:
def make_df(topics):
  topic_list = []
  df_all = []
  for topic in topics:
    topic_list.append("topic {0}".format(topic[0]))
    prob, words = modify_string(topic[1])
    df_topic = pd.DataFrame(data = [prob, words], index = ["Probabilities", "Words"]).T
    df_all.append(df_topic)
  df = pd.concat(df_all, axis = 1, keys = topic_list  )   #https://stackoverflow.com/questions/40820017/how-to-create-a-multilevel-dataframe-in-pandas
  
  return df


**Training with eta = "auto", alpha = "auto", 10 topics, 1000 passes** 






In [ ]:

def test_eta(eta, alpha, dictionary, ntopics, print_topics=True, print_dist=True):
    np.random.seed(42) # set the random seed for repeatability
    bow = [dictionary.doc2bow(doc) for doc in docs] # get the bow-format lines with the set dictionary
    eval_every = None
    import logging
    #logging.basicConfig( filename ='gensim_150_info.log',
     #               format="%(asctime)s:%(levelname)s:%(message)s",
      #              level=logging.INFO,
       #             ) 
    logging.basicConfig(filename='gensim_10_debug.log',
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.DEBUG)
    with (np.errstate(divide='ignore')):  # ignore divide-by-zero warnings
        model = LdaModel(
            corpus=bow, id2word=dictionary, num_topics=ntopics,
             chunksize=2000, eta=eta, random_state=42,
            eval_every= eval_every,   update_every=1,
            passes= 1000, iterations = 400, alpha= alpha, per_word_topics=True ) #, compute_loss=True, callbacks=[callback()])
    
    
    print('Perplexity: {:.2f}'.format(model.log_perplexity(bow)))
    if print_topics:
        # display the top terms for each topic
        for topic in range(ntopics):
            print('Topic {}: {}'.format(topic, [dictionary[w] for w,p in model.get_topic_terms(topic, topn=10)]))
  
    return model

In [ ]:
#10 topics, random state = 42, 1000 passes
model_10 = test_eta("auto", "auto",dictionary,ntopics=10)

Perplexity: -8.06
Topic 0: ['venture', 'investment', 'investor', 'bank', 'entrepreneur', 'innovation', 'price', 'audience', 'reputation', 'founder']
Topic 1: ['identity', 'participant', 'job', 'task', 'emotion', 'culture', 'worker', 'life', 'personal', 'feel']
Topic 2: ['team', 'conflict', 'professional', 'meeting', 'project', 'unit', 'patient', 'innovation', 'occupational', 'expertise']
Topic 3: ['family', 'acquisition', 'space', 'growth', 'successor', 'seller', 'technology', 'history', 'child', 'buyer']
Topic 4: ['ceo', 'corporate', 'exchange', 'director', 'ideology', 'board', 'strategic', 'executive', 'leadership', 'journalist']
Topic 5: ['network', 'tie', 'partner', 'client', 'collaboration', 'exchange', 'party', 'option', 'networking', 'third_party']
Topic 6: ['political', 'activist', 'stakeholder', 'government', 'movement', 'protest', 'local', 'community', 'policy', 'player']
Topic 7: ['logic', 'job', 'worker', 'institutional', 'fund', 'task', 'internal', 'union', 'candidate', 'l

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', model_10.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=model_10, texts=docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
#higher the better


Perplexity:  -8.058100268817043

Coherence Score:  0.43988779861403104


In [ ]:
#save the topic distribution
topics = model_10.print_topics(num_words=20)
df = make_df(topics)
topics_distr = []
for docno, doc in enumerate(corpus):
    topics_distr.append(model_10[doc][0])
all_filenames = [file.split("/")[-1] for file in allfiles]
topic_series = pd.Series(data = topics_distr, index =[f[:-4].replace("_", "/") for f in all_filenames])
with pd.ExcelWriter(path + "best_model/No_Essays/topics_fullcorpus_10_auto_auto.xlsx") as writer:  
    topic_series.to_excel(writer, sheet_name='document_wise_topics')
    df.to_excel(writer, sheet_name='topics_wise_keywords')

In [ ]:
#https://nbviewer.jupyter.org/github/dsquareindia/gensim/blob/280375fe14adea67ce6384ba7eabf362b05e6029/docs/notebooks/topic_coherence_tutorial.ipynb#topic=1&lambda=1&term=
gensimvis.prepare(model_10, corpus, dictionary, sort_topics=False)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.080917  0.019279       1        1  16.490200
1     -0.095390 -0.073978       2        1  13.182401
2     -0.169407 -0.034148       3        1  15.697027
3     -0.109844  0.042152       4        1   8.716969
4      0.112719 -0.043225       5        1   8.833061
5      0.093132 -0.089037       6        1   8.399133
6      0.077615  0.270307       7        1   7.340609
7      0.087908 -0.052640       8        1   7.092219
8      0.117787 -0.065527       9        1   9.041695
9     -0.195437  0.026818      10        1   5.206686, topic_info=           Term         Freq        Total Category  logprob  loglift
120         ceo  4167.000000  4167.000000  Default  30.0000  30.0000
516     network  2910.000000  2910.000000  Default  29.0000  29.0000
885       woman  2263.000000  2263.000000  Default  28.0000  28.0000
469       logic  2153.000000  2153.000000  Default  27.0000  27.0000
808        team  3804.000000  3804.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
397   incentive   216.248775   751.595232  Topic10  -5.4462   1.7095
1136   conflict   277.948758  2041.078730  Topic10  -5.1952   0.9614
606     project   246.246699  1524.307111  Topic10  -5.3163   1.1323
1172       core   195.468951   693.750785  Topic10  -5.5473   1.6885
808        team   188.583642  3804.800353  Topic10  -5.5831  -0.0493

[722 rows x 6 columns], token_table=       Topic      Freq                   Term
term                                         
9994       5  0.997662            accelerator
4290       4  0.990294               acquirer
4290       5  0.006430               acquirer
4291       4  0.986463        acquirer_target
906        1  0.082903            acquisition
...      ...       ...                    ...
887        8  0.540408                 worker
887        9  0.061147                 worker
887       10  0.019831                 worker
5791       5  0.978522  workplace_interaction
12240      9  0.986174             xenophobic

[2287 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

10 Topics - random state 43

In [ ]:

def test_eta(eta, alpha, dictionary, ntopics, print_topics=True, print_dist=True):
    np.random.seed(43) # set the random seed for repeatability
    bow = [dictionary.doc2bow(doc) for doc in docs] # get the bow-format lines with the set dictionary
    eval_every = None
    import logging
    #logging.basicConfig( filename ='gensim_150_info.log',
     #               format="%(asctime)s:%(levelname)s:%(message)s",
      #              level=logging.INFO,
       #             ) 
    logging.basicConfig(filename='gensim_10_debug.log',
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.DEBUG)
    with (np.errstate(divide='ignore')):  # ignore divide-by-zero warnings
        model = LdaModel(
            corpus=bow, id2word=dictionary, num_topics=ntopics,
             chunksize=2000, eta=eta, random_state=43,
            eval_every= eval_every,   update_every=1,
            passes= 1000, iterations = 400, alpha= alpha, per_word_topics=True ) #, compute_loss=True, callbacks=[callback()])
    
    
    print('Perplexity: {:.2f}'.format(model.log_perplexity(bow)))
    if print_topics:
        # display the top terms for each topic
        for topic in range(ntopics):
            print('Topic {}: {}'.format(topic, [dictionary[w] for w,p in model.get_topic_terms(topic, topn=10)]))
  
    return model

In [ ]:
#10 topics, random state = 43, 1000 passes
model_10_rs43 = test_eta("auto", "auto",dictionary,ntopics=10)

Perplexity: -8.06
Topic 0: ['political', 'bank', 'acquisition', 'institutional', 'government', 'country', 'stakeholder', 'growth', 'target', 'policy']
Topic 1: ['team', 'meeting', 'expertise', 'project', 'patient', 'occupational', 'unit', 'engineer', 'learning', 'doctor']
Topic 2: ['identity', 'professional', 'participant', 'interview', 'community', 'life', 'personal', 'feel', 'provider', 'respect']
Topic 3: ['creative', 'cultural', 'culture', 'technology', 'innovation', 'collaboration', 'family', 'evaluation', 'incumbent', 'norm']
Topic 4: ['job', 'conflict', 'exchange', 'candidate', 'worker', 'task', 'hiring', 'client', 'newcomer', 'hire']
Topic 5: ['logic', 'routine', 'institutional', 'innovation', 'aspiration', 'option', 'conflict', 'element', 'brokerage', 'history']
Topic 6: ['woman', 'gender', 'female', 'men', 'male', 'participant', 'family', 'country', 'inequality', 'job']
Topic 7: ['ceo', 'corporate', 'director', 'board', 'ideology', 'political', 'executive', 'leadership', 'str

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', model_10_rs43.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=model_10_rs43, texts=docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
#higher the better


Perplexity:  -8.057134272495219

Coherence Score:  0.4313437844952631


In [ ]:
#save the topic distribution
topics = model_10_rs43.print_topics(num_words=20)
df = make_df(topics)
topics_distr = []
for docno, doc in enumerate(corpus):
    topics_distr.append(model_10_rs43[doc][0])
all_filenames = [file.split("/")[-1] for file in allfiles]
topic_series = pd.Series(data = topics_distr, index =[f[:-4].replace("_", "/") for f in all_filenames])
with pd.ExcelWriter(path + "best_model/No_Essays/topics_rs43_fullcorpus_10_auto_auto.xlsx") as writer:  
    topic_series.to_excel(writer, sheet_name='document_wise_topics')
    df.to_excel(writer, sheet_name='topics_wise_keywords')

In [ ]:
#https://nbviewer.jupyter.org/github/dsquareindia/gensim/blob/280375fe14adea67ce6384ba7eabf362b05e6029/docs/notebooks/topic_coherence_tutorial.ipynb#topic=1&lambda=1&term=
gensimvis.prepare(model_10_rs43, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.133486  0.035419       1        1  17.748837
8      0.108057 -0.054955       2        1  11.544062
1     -0.226162  0.020409       3        1  11.040863
9      0.051166 -0.052005       4        1  10.325935
0      0.104391 -0.174022       5        1   9.175619
6      0.066871  0.151232       6        1   8.966096
4      0.034456  0.166347       7        1   8.620464
7      0.158898  0.027466       8        1   7.936762
3     -0.022029 -0.009517       9        1   7.373924
5     -0.142161 -0.110374      10        1   7.267437, topic_info=          Term         Freq        Total Category  logprob  loglift
120        ceo  4164.000000  4164.000000  Default  30.0000  30.0000
808       team  3795.000000  3795.000000  Default  29.0000  29.0000
431        job  3318.000000  3318.000000  Default  28.0000  28.0000
885      woman  2262.000000  2262.000000  Default  27.0000  27.0000
469      logic  2154.000000  2154.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
2373       day   194.730375  1104.139704  Topic10  -5.8845   0.8866
2065      unit   197.507164  1508.673326  Topic10  -5.8703   0.5886
246      elite   181.937197   708.129529  Topic10  -5.9525   1.2628
418   internal   178.339844  1398.180148  Topic10  -5.9724   0.5625
606    project   177.902446  1523.325829  Topic10  -5.9749   0.4744

[728 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
9994       3  0.815373    accelerator
9994       7  0.182698    accelerator
9065       1  0.973085  accommodation
9065       5  0.011058  accommodation
4290       5  0.996356       acquirer
...      ...       ...            ...
3758       1  0.214096      workplace
3758       6  0.500766      workplace
3758       7  0.255826      workplace
3758       9  0.029030      workplace
12750      3  0.991811     young_firm

[2361 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 9, 2, 10, 1, 7, 5, 8, 4, 6])

In [ ]:
40/220

0.18181818181818182

**Training with 13 topics, 1000 passes** 


In [ ]:
#13 topics, random state = 42, 1000 passes
model_13 = test_eta("auto", "auto",dictionary,ntopics=13)

Perplexity: -8.02
Topic 0: ['venture', 'fund', 'audience', 'price', 'investment', 'reputation', 'technology', 'entrepreneur', 'capital', 'innovation']
Topic 1: ['identity', 'participant', 'interview', 'informant', 'worker', 'personal', 'job', 'article', 'respect', 'life']
Topic 2: ['team', 'conflict', 'meeting', 'project', 'patient', 'voice', 'task', 'doctor', 'team_member', 'frame']
Topic 3: ['acquisition', 'target', 'growth', 'seller', 'claim', 'technology', 'buyer', 'acquirer', 'deference', 'beer']
Topic 4: ['ceo', 'director', 'exchange', 'corporate', 'board', 'ideology', 'strategic', 'leadership', 'executive', 'journalist']
Topic 5: ['network', 'participant', 'sustainability', 'activist', 'innovation', 'partner', 'collaboration', 'adoption', 'option', 'fear']
Topic 6: ['player', 'diversity', 'racial', 'candidate', 'improvisation', 'capability', 'campaign', 'commitment', 'income', 'inequality']
Topic 7: ['professional', 'identity', 'occupational', 'provider', 'engineer', 'commander'

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', model_13.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model= model_13, texts=docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
#higher the better


Perplexity:  -8.019509661359542

Coherence Score:  0.41918698309815344


In [ ]:
#https://nbviewer.jupyter.org/github/dsquareindia/gensim/blob/280375fe14adea67ce6384ba7eabf362b05e6029/docs/notebooks/topic_coherence_tutorial.ipynb#topic=1&lambda=1&term=
gensimvis.prepare(model_13, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.110705 -0.064520       1        1  12.682539
10     0.072812 -0.104617       2        1  11.350891
1     -0.155624  0.062928       3        1  10.042559
2     -0.170632  0.002217       4        1   9.259618
8      0.154018  0.012400       5        1   8.871958
7     -0.210587  0.027348       6        1   8.425210
4      0.114802 -0.046390       7        1   7.450719
11    -0.006456  0.055029       8        1   6.862104
5     -0.027222 -0.058453       9        1   6.566085
12     0.112984  0.107271      10        1   6.292969
3     -0.013865 -0.089658      11        1   5.192980
9     -0.044609 -0.123441      12        1   4.404042
6      0.063673  0.219885      13        1   2.598327, topic_info=             Term         Freq        Total Category  logprob  loglift
120           ceo  4173.000000  4173.000000  Default  30.0000  30.0000
431           job  3316.000000  3316.000000  Default  29.0000  29.0000
808          team  3797.000000  3797.000000  Default  28.0000  28.0000
133        client  1852.000000  1852.000000  Default  27.0000  27.0000
387      identity  3436.000000  3436.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
547   participant   160.526493  2401.712664  Topic13  -5.0491   0.9448
431           job   147.485880  3316.155229  Topic13  -5.1338   0.5375
253    employment   115.316173   950.359222  Topic13  -5.3799   1.5411
3035       school   110.549115   771.122104  Topic13  -5.4221   1.7079
552    perception   109.431053   949.585185  Topic13  -5.4323   1.4896

[904 rows x 6 columns], token_table=       Topic      Freq             Term
term                                   
8524       4  0.975764        abductive
9994       7  0.996350      accelerator
4290       9  0.003236         acquirer
4290      11  0.993360         acquirer
4291      11  0.962008  acquirer_target
...      ...       ...              ...
2120      10  0.804835        workforce
12240      8  0.960562       xenophobic
12240     12  0.028252       xenophobic
3268       3  0.974862             yard
12750      1  0.985847       young_firm

[3518 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 11, 2, 3, 9, 8, 5, 12, 6, 13, 4, 10, 7])

**12 Topics**

In [ ]:
#12 topics, random state = 42, 1000 passes
model_12 = test_eta("auto", "auto",dictionary,ntopics=12)

Perplexity: -8.03
Topic 0: ['venture', 'investment', 'fund', 'audience', 'investor', 'innovation', 'price', 'entrepreneur', 'reputation', 'technology']
Topic 1: ['identity', 'professional', 'interview', 'participant', 'life', 'job', 'personal', 'informant', 'respect', 'day']
Topic 2: ['team', 'conflict', 'meeting', 'project', 'professional', 'patient', 'expertise', 'innovation', 'voice', 'task']
Topic 3: ['acquisition', 'target', 'sustainability', 'claim', 'growth', 'technology', 'seller', 'threat', 'space', 'activist']
Topic 4: ['ceo', 'director', 'exchange', 'corporate', 'board', 'ideology', 'strategic', 'leadership', 'executive', 'journalist']
Topic 5: ['network', 'tie', 'partner', 'collaboration', 'party', 'client', 'exchange', 'networking', 'adoption', 'third_party']
Topic 6: ['player', 'participant', 'improvisation', 'option', 'candidate', 'racial', 'capability', 'alessi', 'creator', 'audience']
Topic 7: ['job', 'worker', 'task', 'occupational', 'candidate', 'labor', 'internal', 

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', model_12.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model= model_12, texts=docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
#higher the better


Perplexity:  -8.025468807515413

Coherence Score:  0.43153124492534084


In [ ]:
#save the topic distribution
topics = model_12.print_topics(num_words=20)
df = make_df(topics)
topics_distr = []
for docno, doc in enumerate(corpus):
    topics_distr.append(model_12[doc][0])
all_filenames = [file.split("/")[-1] for file in allfiles]
topic_series = pd.Series(data = topics_distr, index =[f[:-4].replace("_", "/") for f in all_filenames])
with pd.ExcelWriter(path + "best_model/No_Essays/topics_fullcorpus_12_auto_auto.xlsx") as writer:  
    topic_series.to_excel(writer, sheet_name='document_wise_topics')
    df.to_excel(writer, sheet_name='topics_wise_keywords')

In [ ]:
#https://nbviewer.jupyter.org/github/dsquareindia/gensim/blob/280375fe14adea67ce6384ba7eabf362b05e6029/docs/notebooks/topic_coherence_tutorial.ipynb#topic=1&lambda=1&term=
gensimvis.prepare(model_12, corpus, dictionary, sort_topics=False)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.102140  0.029222       1        1  13.776581
1      0.193227  0.030233       2        1  12.084950
2      0.181755  0.051615       3        1  13.141946
3      0.051733  0.126959       4        1   6.702587
4     -0.105146 -0.012311       5        1   7.784631
5     -0.092489  0.033046       6        1   7.450286
6      0.056366 -0.169298       7        1   3.324371
7      0.007976 -0.128676       8        1   5.594903
8     -0.116752 -0.114206       9        1   7.954610
9     -0.032190  0.118521      10        1   4.473752
10    -0.103186  0.110448      11        1  10.381695
11     0.060846 -0.075552      12        1   7.329686, topic_info=             Term         Freq        Total Category  logprob  loglift
120           ceo  4171.000000  4171.000000  Default  30.0000  30.0000
516       network  2908.000000  2908.000000  Default  29.0000  29.0000
808          team  3807.000000  3807.000000  Default  28.0000  28.0000
387      identity  3441.000000  3441.000000  Default  27.0000  27.0000
431           job  3312.000000  3312.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
2915     powerful   207.250388   433.770623  Topic12  -5.8307   1.8746
849   uncertainty   228.841517   795.386776  Topic12  -5.7316   1.3674
1541         item   224.055044   720.110423  Topic12  -5.7528   1.4457
805          task   263.130273  2225.058488  Topic12  -5.5920   0.4783
2529   experiment   211.941074   581.272514  Topic12  -5.8083   1.6043

[835 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
10945      2  0.977049           ac
9994       5  0.996544  accelerator
4290       4  0.992106     acquirer
4290      11  0.003221     acquirer
906        1  0.055049  acquisition
...      ...       ...          ...
5532       3  0.367325     workshop
5532       4  0.079853     workshop
5532       7  0.537678     workshop
5532      12  0.010647     workshop
12750      1  0.986023   young_firm

[3112 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

**Guided LDA**


Giving seed words as priors

In [ ]:
def create_eta(priors, etadict, ntopics):
    eta = np.full(shape=(ntopics, len(etadict)), fill_value=1) # create a (ntopics, nterms) matrix and fill with 1
    for word, topic in priors.items(): # for each word in the list of priors
        keyindex = [index for index,term in etadict.items() if term==word] # look up the word in the dictionary
        if (len(keyindex)>0): # if it's in the dictionary
            eta[topic,keyindex[0]] = 1e7  # put a large number in there
    eta = np.divide(eta, eta.sum(axis=0)) # normalize so that the probabilities sum to 1 over all topics
    return eta

In [ ]:
apriori_modified = {
    'woman':9, 'female':9, 'gender':9,
    'director':1, 'board':1, 'executive':1,
    'professional':3, 'occupational':3,
    'partner': 4, 'client' : 4,
    'conflict' : 5,
    'political': 6, 'institutional': 6,
    'innovation' : 7,
    'idea' : 8, 'voice' : 8, 
    'entreprenuer':0, 'venture':0, 'network':0,
    'protest' : 2, 'activist':2, 'movement':2

}
eta = create_eta(apriori_modified, dictionary, 10)
#model_guided = test_eta(eta, dictionary, 11)

In [ ]:
#change log filename in test_eta before running this 
#10 topics, 1000 passes, random state 42
model_guided = test_eta(eta, "auto",  dictionary, 10)

Perplexity: -6.75
Topic 0: ['network', 'venture', 'investor', 'entrepreneur', 'investment', 'label', 'candidate', 'founder', 'logic', 'community']
Topic 1: ['director', 'corporate', 'board', 'reputation', 'analyst', 'hypothesis', 'executive', 'investor', 'journalist', 'focal']
Topic 2: ['community', 'activist', 'ideology', 'movement', 'political', 'local', 'adoption', 'green', 'boundary', 'institutional']
Topic 3: ['identity', 'professional', 'participant', 'interview', 'occupational', 'role', 'meeting', 'learning', 'patient', 'personal']
Topic 4: ['partner', 'client', 'exchange', 'network', 'logic', 'collaboration', 'target', 'stakeholder', 'party', 'project']
Topic 5: ['conflict', 'routine', 'logic', 'founder', 'player', 'temporal', 'team', 'origin', 'bank', 'institutional']
Topic 6: ['political', 'country', 'government', 'institutional', 'local', 'financial', 'policy', 'economic', 'entry', 'entrepreneur']
Topic 7: ['innovation', 'technology', 'cultural', 'technological', 'search', '

In [ ]:
#save the topic distribution
topics = model_guided.print_topics(num_words=20)
df = make_df(topics)
topics_distr = []
for docno, doc in enumerate(corpus):
    topics_distr.append(model_guided[doc][0])
all_filenames = [file.split("/")[-1] for file in allfiles]
topic_series = pd.Series(data = topics_distr, index =[f[:-4].replace("_", "/") for f in all_filenames])
with pd.ExcelWriter(path + "best_model/topics_guided.xlsx") as writer:  
    topic_series.to_excel(writer, sheet_name='document_wise_topics')
    df.to_excel(writer, sheet_name='topics_wise_keywords')

In [ ]:
#https://medium.com/analytics-vidhya/topic-modeling-using-gensim-lda-in-python-48eaa2344920
# Compute Perplexity
print('\nPerplexity: ',model_guided.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=model_guided, texts=docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.750936769098025

Coherence Score:  0.38691547860530495


In [ ]:
#https://nbviewer.jupyter.org/github/dsquareindia/gensim/blob/280375fe14adea67ce6384ba7eabf362b05e6029/docs/notebooks/topic_coherence_tutorial.ipynb#topic=1&lambda=1&term=
gensimvis.prepare(model_guided, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.154243 -0.084623       1        1  18.875240
9     -0.105114 -0.144424       2        1  10.885055
6     -0.134672  0.154860       3        1  10.755521
1     -0.155405 -0.012834       4        1  10.209494
2      0.103027  0.111351       5        1   9.482745
7      0.027478 -0.005543       6        1   9.148247
0     -0.005180  0.021948       7        1   8.724486
4     -0.069196 -0.002842       8        1   7.618481
8      0.045561 -0.113202       9        1   7.453058
5      0.139259  0.075308      10        1   6.847674, topic_info=             Term         Freq        Total Category  logprob  loglift
1138        woman  2269.000000  2269.000000  Default  30.0000  30.0000
3066      partner  1867.000000  1867.000000  Default  29.0000  29.0000
156        client  1847.000000  1847.000000  Default  28.0000  28.0000
1508       gender  1679.000000  1679.000000  Default  27.0000  27.0000
2325      network  2892.000000  2892.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
2173    expertise   225.892332  1089.860152  Topic10  -5.5847   1.1075
3092     pressure   206.765532   812.674443  Topic10  -5.6731   1.3125
2628      banking   172.302719   447.033205  Topic10  -5.8555   1.7279
1059  traditional   164.104201   681.092551  Topic10  -5.9042   1.2581
1873    strategic   157.256221  1133.783397  Topic10  -5.9468   0.7058

[736 rows x 6 columns], token_table=       Topic      Freq             Term
term                                   
11376      1  0.996190      accelerator
4979       3  0.010702         acquirer
4979       8  0.984577         acquirer
4980       2  0.031024  acquirer_target
4980       8  0.930723  acquirer_target
...      ...       ...              ...
1142       3  0.006003           worker
1142       5  0.034929           worker
1142       6  0.025651           worker
1142       8  0.018556           worker
10677      1  0.985810        workspace

[2456 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 10, 7, 2, 3, 8, 1, 5, 9, 6])

Guided, 1000 passes, 13 topics

In [ ]:
#change log filename in test_eta before running this 
#14 topics, 500 passes, random state 43
eta = create_eta(apriori_modified, dictionary, 13)
model_13_guided = test_eta(eta, "auto", dictionary, 13)

Perplexity: -6.26
Topic 0: ['network', 'logic', 'community', 'family', 'domain', 'closure', 'cultural', 'history', 'alessi', 'price']
Topic 1: ['country', 'political', 'financial', 'investor', 'institutional', 'investment', 'venture', 'government', 'entrepreneur', 'incentive']
Topic 2: ['community', 'local', 'adoption', 'boundary', 'activist', 'leader', 'school', 'campaign', 'diversity', 'chemistry']
Topic 3: ['occupational', 'occupation', 'learning', 'client', 'professional', 'worker', 'segregation', 'nurse', 'socialization', 'venture']
Topic 4: ['partner', 'client', 'network', 'collaboration', 'exchange', 'project', 'target', 'engineer', 'party', 'acquisition']
Topic 5: ['conflict', 'routine', 'founder', 'bank', 'temporal', 'origin', 'elite', 'magazine', 'improvisation', 'team']
Topic 6: ['exchange', 'stakeholder', 'protest', 'subsidiary', 'union', 'article', 'target', 'entrepreneur', 'ritual', 'generalized']
Topic 7: ['innovation', 'technology', 'cultural', 'technological', 'journal

In [ ]:
#save the topic distribution
topics = model_13_guided.print_topics(num_words=20)
df = make_df(topics)
topics_distr = []
for docno, doc in enumerate(corpus):
    topics_distr.append(model_13_guided[doc][0])
all_filenames = [file.split("/")[-1] for file in allfiles]
topic_series = pd.Series(data = topics_distr, index =[f[:-4].replace("_", "/") for f in all_filenames])
with pd.ExcelWriter(path + "best_model/13topics_guided.xlsx") as writer:  
    topic_series.to_excel(writer, sheet_name='document_wise_topics')
    df.to_excel(writer, sheet_name='topics_wise_keywords')

In [ ]:
#https://medium.com/analytics-vidhya/topic-modeling-using-gensim-lda-in-python-48eaa2344920
# Compute Perplexity
print('\nPerplexity: ', model_13_guided.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=model_13_guided, texts=docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.263784972935585

Coherence Score:  0.3700309773637861


In [ ]:
#https://nbviewer.jupyter.org/github/dsquareindia/gensim/blob/280375fe14adea67ce6384ba7eabf362b05e6029/docs/notebooks/topic_coherence_tutorial.ipynb#topic=1&lambda=1&term=
gensimvis.prepare(model_13_guided, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.135598 -0.052363       1        1  14.754466
1      0.123969  0.104078       2        1  12.831595
10     0.152121 -0.019184       3        1   8.751645
8      0.091851 -0.145616       4        1   7.986082
9      0.144721 -0.139871       5        1   7.695850
2     -0.076880  0.065503       6        1   7.204238
7     -0.048566  0.028527       7        1   7.115466
4      0.047986  0.013530       8        1   5.991939
3     -0.165230 -0.047048       9        1   5.938409
11    -0.138597 -0.107192      10        1   5.816698
0     -0.010571  0.072652      11        1   5.804510
6      0.124974  0.108235      12        1   5.134725
5     -0.110180  0.118751      13        1   4.974377, topic_info=            Term         Freq        Total Category  logprob  loglift
535     identity  3316.000000  3316.000000  Default  30.0000  30.0000
2325     network  2892.000000  2892.000000  Default  29.0000  29.0000
1138       woman  2259.000000  2259.000000  Default  28.0000  28.0000
3066     partner  1869.000000  1869.000000  Default  27.0000  27.0000
1328    conflict  2014.000000  2014.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
1235   authority   145.929296   691.002341  Topic13  -5.7020   1.4458
183   commercial   120.917606   367.043203  Topic13  -5.8900   1.8905
553     indirect   124.698496   487.364192  Topic13  -5.8592   1.6378
3531       frame   130.711039   772.599069  Topic13  -5.8121   1.2241
914         role   136.324047  1255.919200  Topic13  -5.7701   0.7803

[937 rows x 6 columns], token_table=       Topic      Freq                   Term
term                                         
11376      1  0.003413            accelerator
11376      7  0.098968            accelerator
11376      9  0.651823            accelerator
11376     12  0.242301            accelerator
4979       3  0.096078               acquirer
...      ...       ...                    ...
1142      10  0.017544                 worker
1142      12  0.003289                 worker
9486       5  0.982169        worker_entering
1962       4  0.055943  workplace_interaction
1962      12  0.932378  workplace_interaction

[3596 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 2, 11, 9, 10, 3, 8, 5, 4, 12, 1, 7, 6])